In [13]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import math

In [14]:
class Selayer(nn.Module):

    def __init__(self, inplanes):
        super(Selayer, self).__init__()
        self.global_avgpool = nn.AdaptiveAvgPool2d(1)
        self.conv1 = nn.Conv2d(int(inplanes), int(inplanes / 16), kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(int(inplanes / 16), int(inplanes), kernel_size=1, stride=1)
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        out = self.global_avgpool(x)

        out = self.conv1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.sigmoid(out)

        return x * out
    
class BottleneckX(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, cardinality, stride=1, downsample=None):
        super(BottleneckX, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes * 2)

        self.conv2 = nn.Conv2d(planes * 2, planes * 2, kernel_size=3, stride=stride,
                               padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm2d(planes * 2)

        self.conv3 = nn.Conv2d(planes * 2, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)

        self.selayer = Selayer(planes * 4)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out = self.selayer(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
class SEResNeXt(nn.Module):

    def __init__(self, block, layers, cardinality=32, num_classes=10):
        super(SEResNeXt, self).__init__()
        self.cardinality = cardinality
        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, self.cardinality, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, self.cardinality))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        x = self.fc(x)

        return x


In [20]:
data_path = "C://Users//shjdl//Desktop//DM_project//data//"
test_path = data_path+"test.csv"
SEResNeXt(BottleneckX, [3, 4, 6, 3],cardinality=32, num_classes=10)
model_path = "./model-2019.04.08.pt"
model=torch.load(model_path)
model.eval()

test_raw_data = pd.read_csv(test_path, iterator=True)
test_data = test_raw_data.get_chunk(10000).values.astype('int')

In [16]:
print(test_data.shape)
print(test_data[600])

(10000, 3073)
[600  57  57 ...  13  11  12]


In [21]:
class TestDataset(data.Dataset):
    def __init__(self, test_data):
        self.data=test_data
    def __len__(self):
        return 10000
    def __getitem__(self, idx):
        img = self.data[idx]
        img_id = img[0]
        img = img[1:3073]
        img = np.array(img).reshape(3,32,32)
        img = img/256
        return img,img_id

In [22]:
datetime =time.strftime('%Y.%m.%d',time.localtime(time.time()))
result_path = data_path+"result-" +datetime+ ".csv"
test_dataset =  TestDataset(test_data)
test_loader = data.DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False)
out = open(result_path,'a', newline='')
csv_write = csv.writer(out)
csv_write.writerow(["Id","Category"])

13

In [23]:
result = []
for image,imageid in tqdm(test_loader,leave=False,disable=False):
    image = image.type(torch.FloatTensor)
    if torch.cuda.is_available():
        image = image.cuda()
        model = model.cuda()
    out = model(image)
    pred = torch.max(out, 1)[1]
    result.append([int(imageid) , pred.item()])


  5%|████▏                                                                         | 536/10000 [00:16<04:48, 32.81it/s]


 11%|████████▎                                                                    | 1080/10000 [00:32<04:18, 34.52it/s]


 16%|████████████▌                                                                | 1624/10000 [00:48<04:22, 31.86it/s]


 22%|████████████████▋                                                            | 2168/10000 [01:04<03:48, 34.24it/s]


 27%|████████████████████▉                                                        | 2712/10000 [01:20<03:18, 36.79it/s]


...

 82%|██████████████████████████████████████████████████████████████▊              | 8151/10000 [04:00<00:56, 32.91it/s]


 87%|██████████████████████████████████████████████████████████████████▉          | 8695/10000 [04:15<00:38, 33.54it/s]


 92%|███████████████████████████████████████████████████████████████████████▏     | 9239/10000 [04:31<00:23, 32.30it/s]


 98%|███████████████████████████████████████████████████████████████████████████▎ | 9783/10000 [04:47<00:06, 35.81it/s]


100%|████████████████████████████████████████████████████████████████████████████▉| 9999/10000 [04:54<00:00, 33.34it/s]
                                                                                                                       

In [24]:
print(len(result))

10000


In [8]:
for i in range(0,10000):
    csv_write.writerow(result[i])

In [9]:
for i in range(9361,9800):
    csv_write.writerow(result[i])